In [1]:
import pandas as pd

In [2]:
_bea_gdp_df = pd.read_csv('./data/bea/GDP_2020-2009.csv',dtype={'GeoFips':'string', 'year':'Int64'})
_bea_gdp_df.dropna(inplace=True)
_patents = pd.read_csv('./data/patentsview/all_patents_granted_and pregrant.csv',dtype={'GEOID':'string', 'grant_year':'Int64', 'application_year':'Int64'}, low_memory=False)
_patents.GEOID = _patents.GEOID.str.rjust(5, '0')
# _patents = _patents[_patents['application_year'].notna()]
# _patents = _patents.query("application_year >= 2010", engine='python')
print(_patents.shape)

(1886618, 12)


In [5]:
_bea_gdp_df.query("GeoFips == '51650'", engine='python')

,ID,GeoFips,GeoName,GDP,year
34776,2899.0,51650,"Hampton (Independent City), VA",6500443,2009
34777,6018.0,51650,"Hampton (Independent City), VA",6382910,2010
34778,9137.0,51650,"Hampton (Independent City), VA",6319375,2011
34779,12256.0,51650,"Hampton (Independent City), VA",6218426,2012
34780,15375.0,51650,"Hampton (Independent City), VA",6264366,2013
34781,18494.0,51650,"Hampton (Independent City), VA",6338289,2014
34782,21613.0,51650,"Hampton (Independent City), VA",6329656,2015
34783,24732.0,51650,"Hampton (Independent City), VA",6423896,2016
34784,27851.0,51650,"Hampton (Independent City), VA",6648540,2017
34785,30970.0,51650,"Hampton (Independent City), VA",6674453,2018


In [6]:
_patents.query("GEOID == '51650'", engine='python')

,patent_number,assignee,grant_year,application_year,application_number,GEOID,ipc_section,team_size,inventors,men_inventors,women_inventors,already_granted
1042383,7845643,"CYDEWL, LLC",2010,2009,12322672,51650,A,1,1,1.0,0,1
1042384,7982200,HAMPTON UNIVERSITY,2011,2009,12466979,51650,A,3,3,2.0,1,1
1042385,8133167,HAMPTON UNIVERSITY,2012,2009,12647920,51650,A,4,4,3.0,1,1
1042386,8200316,HAMPTON UNIVERSITY,2012,2010,12688509,51650,A,2,2,1.0,1,1
1042387,8248888,"MEASUREMENT SPECIALTIES, INC.",2012,2010,12850248,51650,G,2,2,2.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1876232,NaN,"Psionic, LLC",<NA>,2019,16501526,51650,NaN,3,3,3.0,0,0
1882910,NaN,"MEASUREMENT SPECIALTIES, INC.",<NA>,2019,16395095,51650,NaN,2,2,2.0,0,0
1882918,NaN,"MEASUREMENT SPECIALTIES, INC.",<NA>,2019,16394915,51650,NaN,2,2,2.0,0,0
1884048,NaN,"MEASUREMENT SPECIALTIES, INC.",<NA>,2019,16434124,51650,NaN,5,5,4.0,0,0


In [3]:
_bea_gdp_df.head()

,ID,GeoFips,GeoName,GDP,year
0,1.0,01001,"Autauga, AL",1179897,2009
1,3120.0,01001,"Autauga, AL",1265180,2010
2,6239.0,01001,"Autauga, AL",1368637,2011
3,9358.0,01001,"Autauga, AL",1587254,2012
4,12477.0,01001,"Autauga, AL",1519900,2013


In [ ]:
# Changing all the combined bea fips to county fips

bea_group_to_census_county_dict = {
'15901':'15005', # 15901 Maui and Kalawao, HI 15005 Kalawao, HI
'51901':'51003', # 51901 Albemarle + Charlottesville, VA  51003 Albemarle, VA 
'51903':'51005', # 51903 Alleghany + Covington, VA  51005 Alleghany, VA 
'51907':'51015', # 51907 Augusta, Staunton + Waynesboro, VA  51015 Augusta, VA 
'51909':'51019', # 51909 Bedford + Bedford City, VA 51019 Bedford, VA
'51911':'51031', # 51911 Campbell + Lynchburg, VA  51031 Campbell, VA 
'51913':'51035', # 51913 Carroll + Galax, VA  51035 Carroll, VA 
'51918':'51053', # 51918 Dinwiddie, Colonial Heights + Petersburg, VA  51053 Dinwiddie, VA 
'51919':'51059', # 51919 Fairfax, Fairfax City + Falls Church, VA  51059 Fairfax, VA 
'51921':'51069', # 51921 Frederick + Winchester, VA  51069 Frederick, VA 
'51923':'51081', # 51923 Greensville + Emporia, VA  51081 Greensville, VA 
'51925':'51083', # 51925 Halifax + South Boston, VA 51083 Halifax, VA
'51929':'51089', # 51929 Henry + Martinsville, VA  51089 Henry, VA 
'51931':'51095', # 51931 James City + Williamsburg, VA  51095 James City, VA 
'51933':'51121', # 51933 Montgomery + Radford, VA  51121 Montgomery, VA 
'51939':'51143', # 51939 Pittsylvania + Danville, VA  51143 Pittsylvania, VA 
'51941':'51149', # 51941 Prince George + Hopewell, VA  51149 Prince George, VA 
'51942':'51153', # 51942 Prince William, Manassas + Manassas Park, VA  51153 Prince William, VA 
'51944':'51161', # 51944 Roanoke + Salem, VA  51161 Roanoke, VA 
'51945':'51163', # 51945 Rockbridge, Buena Vista + Lexington, VA  51163 Rockbridge, VA 
'51947':'51165', # 51947 Rockingham + Harrisonburg, VA  51165 Rockingham, VA 
'51949':'51175', # 51949 Southampton + Franklin, VA  51175 Southampton, VA 
'51951':'51177', # 51951 Spotsylvania + Fredericksburg, VA  51177 Spotsylvania, VA 
'51953':'51191', # 51953 Washington + Bristol, VA  51191 Washington, VA 
'51955':'51195', # 51955 Wise + Norton, VA  51195 Wise, VA 
'51958':'51199', # 51958 York + Poquoson, VA  51199 York, VA 
'55901':'55078', # 55901 Shawano (incl. Menominee), WI (prior to 1989) 55078 Menominee, WI
}

shape = _bea_gdp_df.query('GeoFips == "51901"').shape
print(f'Before replacment: {shape}')
bea_gdp_df = _bea_gdp_df.copy()
bea_gdp_df['original_GeoFips'] = bea_gdp_df['GeoFips']
bea_gdp_df.GeoFips.replace(to_replace=bea_group_to_census_county_dict, inplace=True)
shape = bea_gdp_df.query('GeoFips == "51901"').shape
print(f'After replacment: {shape}')

In [ ]:
# The structure of the dict is: key = independent city
# The values is county the independent city is grouped with by the bea.
# For each key we need to reclassify it to value of the associated county.
census_independent_to_census_county_dict = {
    # '15009':'15005', # 15009 Maui, HI 15005 Kalawao, HI
    '15005':'15009', # 15005 Kalawao, HI, 15009 Maui, HI  ###HAD THIS WRONG...
    '51540':'51003', # 51540 Charlottesville (Independent City), VA  51003 Albemarle, VA 
    '51580':'51005', # 51580 Covington (Independent City), VA  51005 Alleghany, VA 
    '51790':'51015', # 51790 Staunton (Independent City), VA  51015 Augusta, VA 
    '51820':'51015', # 51820 Waynesboro (Independent City), VA  51015 Augusta, VA 
    '51515':'51019', # 51515 Bedford (Indenpedent City), VA 51019 Bedford, VA
    '51680':'51031', # 51680 Lynchburg (Independent City), VA  51031 Campbell, VA 
    '51640':'51035', # 51640 Galax (Independent City), VA  51035 Carroll, VA 
    '51570':'51053', # 51570 Colonial Heights (Independent City), VA  51053 Dinwiddie, VA 
    '51730':'51053', # 51730 Petersburg (Independent City), VA  51053 Dinwiddie, VA 
    '51600':'51059', # 51600 Fairfax City (Independent City), VA  51059 Fairfax, VA 
    '51610':'51059', # 51610 Falls Church (Independent City), VA  51059 Fairfax, VA 
    '51840':'51069', # 51840 Winchester (Independent City), VA  51069 Frederick, VA 
    '51595':'51081', # 51595 Emporia (Independent City), VA  51081 Greensville, VA 
    '51780':'51083', # 51780 South Boston (Independent City), VA 51083 Halifax, VA
    '51690':'51089', # 51690 Martinsville (Independent City), VA  51089 Henry, VA 
    '51830':'51095', # 51830 Williamsburg (Independent City), VA  51095 James City, VA 
    '51750':'51121', # 51750 Radford (Independent City), VA  51121 Montgomery, VA 
    '51590':'51143', # 51590 Danville (Independent City), VA  51143 Pittsylvania, VA 
    '51670':'51149', # 51670 Hopewell (Independent City), VA  51149 Prince George, VA 
    '51683':'51153', # 51683 Manassas (Independent City), VA  51153 Prince William, VA 
    '51685':'51153', # 51685 Manassas Park (Independent City), VA  51153 Prince William, VA 
    '51775':'51161', # 51775 Salem (Independent City), VA  51161 Roanoke, VA 
    '51530':'51163', # 51530 Buena Vista (Independent City), VA  51163 Rockbridge, VA 
    '51678':'51163', # 51678 Lexington (Independent City), VA  51163 Rockbridge, VA 
    '51660':'51165', # 51660 Harrisonburg (Independent City), VA  51165 Rockingham, VA 
    '51620':'51175', # 51620 Franklin (Independent City), VA  51175 Southampton, VA 
    '51630':'51177', # 51630 Fredericksburg (Independent City), VA  51177 Spotsylvania, VA 
    '51520':'51191', # 51520 Bristol (Independent City), VA  51191 Washington, VA 
    '51720':'51195', # 51720 Norton (Independent City), VA  51195 Wise, VA 
    '51735':'51199', # 51735 Poquoson (Independent City), VA  51199 York, VA 
    '55115':'55078' # 55115 Shawano, WI 55078 Menominee, WI
}

shape = _patents.query('GEOID == "51840"').shape
print(f'Before replacment: {shape}')
patents = _patents.copy()
patents['original_GEOID'] = patents['GEOID']
patents.GEOID.replace(to_replace=census_independent_to_census_county_dict, inplace=True)
shape = patents.query('GEOID == "51840"').shape
print(f'After replacment: {shape}')

## Lets check to see how we did.

In [ ]:
# All the unique fips in the bea data
unique_bea_geofips = bea_gdp_df.groupby('GeoFips').sum().reset_index()
print(f'Unique bea geofips: {unique_bea_geofips.shape[0]}')

# All the unique fips in the patents data
unique_county_GEOIDS = patents.groupby('GEOID').sum().reset_index()
print(f'Unique county GEOIDS: {unique_county_GEOIDS.shape[0]}')

In [ ]:
# How many missing values are there in the county_GEOID_cross_df?
# That is to say, how many rows in bea are missing rows in the county crosswalk?
test_merge = pd.merge(unique_county_GEOIDS, unique_bea_geofips,
                      left_on = ['GEOID'],
                      right_on = ['GeoFips'],
                      how = 'outer',
                      indicator = 'matched',
                      validate = 'one_to_one')

print(len(test_merge.query("matched == 'left_only'").GeoFips.unique()))
# The missing values

missing_values_bea = pd.DataFrame(test_merge.query("matched == 'right_only'").groupby(['GeoFips']).size()).reset_index()
missing_values_patents = pd.DataFrame(test_merge.query("matched == 'left_only'").groupby(['GEOID']).size()).reset_index()

In [ ]:
print(missing_values_bea.shape)
missing_values_bea.head()

In [ ]:
print(missing_values_patents.shape)
missing_values_patents
# Looks like these are all from terrirories and not counties.

## Some spot checks

In [ ]:
# Should we just get rid of these rows?
# patents_clean = patents[~patents.GEOID.str.startswith(('78', '72' , '69','66' ))].copy()
patents_clean = patents.copy()

In [ ]:
# Merge bea gdp and patents data
test_merge = pd.merge(patents_clean, bea_gdp_df,
                      left_on = ['GEOID', 'application_year'],
                      right_on = ['GeoFips', 'year'],
                      how = 'outer',
                      indicator = 'matched',
                      validate = 'many_to_one')

print(len(test_merge.query("matched == 'left_only'").GeoFips.unique()))
# The missing values
missing_values_patents = pd.DataFrame(test_merge.query("matched == 'left_only'")).reset_index()

In [ ]:
# Drop the terrirory rows
missing_values_patents = missing_values_patents[~missing_values_patents.GEOID.str.startswith(('78', '72' , '69','66' ))]
# We don't have any bea data prior to 2010 so drop those rows as well.
missing_values_patents.query("application_year > 2009", engine='python')

In [ ]:
test_merge.shape

## Do the Merge

In [ ]:
final_merge = pd.merge(patents, bea_gdp_df,
                      left_on = ['GEOID', 'application_year'],
                      right_on = ['GeoFips', 'year'],
                      how = 'inner')

In [ ]:
final_merge.shape

## Check some of the data

In [ ]:
_patents.query("GEOID == '51540' & application_year > 2009", engine='python').head() # 51540 Charlottesville (Independent City), VA  51003 Albemarle, VA 

In [ ]:
final_merge.query("GEOID == '51540'").head() # 51540 Charlottesville (Independent City), VA  51003 Albemarle, VA 

In [ ]:
final_merge.query("patent_number == '7919527'").head() # Check a patent number from above query

In [ ]:
_bea_gdp_df.query("GeoFips == '51901' & year > 2009", engine='python').head(11) # Check a bea gdp row from above query

In [ ]:
bea_gdp_df.query("GeoFips == '51003' & year > 2009", engine='python').head(11) # Check a bea gdp row from above query

## Save to csv


In [ ]:
final_merge.to_csv('./data/bea/bea_gdp_to_patent.csv')